In [1]:
import os
from django.db.models import Q, F, When, Case
from django.db.models import Value, BooleanField, DecimalField

In [2]:
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

# Carts

In [3]:
from cart import models
from shop import models as shop_models

In [4]:
carts = models.Cart.objects.all()

In [5]:
cart = carts.filter(cart_id="2020_9_13_de42d5c80c8a4bada51fcd02")

# Logic to use the discounted price of a product if its discounted
# or its normal price if its not
logic = (
    Q(product__discounted_price__gt=0) & 
    Q(product__discounted=True)
)

condition = When(logic, then='product__discounted_price')
queryset = cart.annotate(true_price=Case(condition, default='product__price_ht'))

condition = When(true_price__gt=0, then=F('true_price') * F('quantity'))
queryset = queryset.annotate(total=Case(condition, output_field=DecimalField()))

## Adding an item to the cart

In [39]:
cart_id = "2020_9_15_cd4ec02a2517741db583a6f9"

quantity = 1
color = "Blanc"
size = None

new_quantity = 1

product = shop_models.Product.objects.first()

# Test if the cart exists with the same product BUT not with the same color and size
characteristics = (
    Q(color=color) &
    Q(size=size)
)

other = (
    Q(product__id=product.id) & 
    Q(cart_id__iexact=cart_id)
)

logic = characteristics & other
items = models.Cart.objects.filter(logic)
if items.exists():
    item = items.get()
    item.price_ht = product.price_ht
    item.product = product
    item.color = color
    item.size = size
    item.quantity = F('quantity') + int(new_quantity)
    item.anonymous = False
    item.save()
else:
    details = {
        'cart_id': cart_id,
        'price_ht': product.get_price(),
        'size': size,
        'color': color,
        'quantity': int(quantity),
        'anonymous': True,
        'product': product
    }
    models.Cart.objects.create(**details)

# Impressions

In [20]:
import json

In [42]:
def create_impressions(queryset, include_position=False):
    impressions = list(
        queryset.values(
            'product__reference', 
            'product__name', 
            'product__collection__name', 
            'true_price', 
            'quantity'
        )
    )
    fit_transformed = []
    fit_values = {
        'product__reference': 'id',
        'product__name': 'name',
        'product__collection__name': 'category',
        'true_price': 'price',
        'quantity': 'quantity',
        'brand': 'brand',
        'position': 'position'
    }
    for index, impression in enumerate(impressions, start=1):
        new_values = {}
        impression['brand'] = 'Nawoka'
        if include_position:
            impression['position'] = index
        for key, value in impression.items():
            new_values[fit_values[key]] = str(value)
        fit_transformed.append(new_values)
    return json.dumps(fit_transformed)

create_impressions(queryset, include_position=True)

'[{"id": "ZL7025928032C1", "name": "Hailey Baldwin", "category": "Lingerie", "quantity": "3", "price": "62.40", "brand": "Nawoka", "position": "1"}, {"id": "FW479666FE1C5C", "name": "Kendall Jenner", "category": "Lingerie", "quantity": "1", "price": "35.00", "brand": "Nawoka", "position": "2"}]'

In [11]:
from shop import models
import re

In [12]:
product = models.Product.objects.first()

In [40]:
s = 'price:get_price'
re.match(r'^(?P<name>\w+)\:(?P<func>.*)$', s).groupdict()

{'name': 'price', 'func': 'get_price'}

[1, 2, 3, 6, 7]

In [6]:
Product.objects.values('product__get_absolute_url')

FieldError: Cannot resolve keyword 'product' into field. Choices are: active, cart, collection, collection_id, created_on, description, description_html, description_objects, discount, discount_pct, discounted, discounted_price, gender, google_category, id, images, in_stock, last_modified, like, lookbook, name, our_favorite, price_pre_tax, price_valid_until, private, quantity, reference, review, sku, slug, to_be_published_on, variant